# Image Manipulation with OpenCV and NumPy Deployed on [Streamlit](https://bit.ly/3qNybNt)

## Introduction

### Import Libraries

In [1]:
import cv2
import numpy as np
import math

### Define `image_resize()` to resize image

In [2]:
def image_resize(image):
    scale_factor = int(input("Enter the scale factor by which to reduce the image size:"))
    height = int(image.shape[0] * scale_factor/100)
    width = int(image.shape[1] * scale_factor/100)
    dimensions = (width,height)

    return cv2.resize(image, dimensions)

### Define `is_outside()` to test limits of image data

In [3]:
def is_outside(r, c, shape):
    if(c < 0 or c >= shape[1]):
        return True
    if(r < 0 or r >= shape[0]):
        return True
    else:
        return False

### Read in the image.

In [33]:
path = "../../media/images/"
file_name = input("Enter file name to use:")
image_file = cv2.imread(path+file_name)

input_image = image_resize(image_file)

Enter file name to use:fly.jpg
Enter the scale factor by which to reduce the image size:100


In [17]:
print(image_file[:2,:2])

[[[109 122 136]
  [107 123 136]]

 [[110 123 137]
  [107 120 134]]]


### Display image

In [37]:
cv2.imshow("Image",input_image)
while True:
        if(cv2.waitKey(1) & 0xFF == ord("q")):
            break
cv2.destroyAllWindows()

### Define `show_image()` to display image

In [6]:
def show_image(name,image):
    cv2.imshow(name,image)
    while True:
        if(cv2.waitKey(1) & 0xFF == ord("q")):
            break
    cv2.destroyAllWindows()

In [7]:
show_image("Image", input_image)

### Write Image to File

In [8]:
cv2.imwrite(path+"resized.jpg", input_image)

True

### Display shape information of image (number of rows, number of columns and number of channels)

In [9]:
input_image.shape

(536, 800, 3)

### Display a slice of the image
#### Full Colour

In [10]:
show_image("Image",input_image[200:400,200:400])

#### One Channel Colour

In [11]:
show_image("Image",input_image[200:400,200:400,1])

### Make a contrived image to play around with.

In [12]:
height = 500
width = 500
channels = 3
contrived_image = (np.random.rand(height,width,channels) * 10).round(2)

### Two ways of displaying value of a pixel of particular channel

In [13]:
contrived_image[:1, :1]

array([[[3.72, 4.9 , 2.33]]])

In [14]:
contrived_image[1,2,1], contrived_image.item((1,2,1))

(8.78, 8.78)

### Two ways of setting the value of a pixel of a particular channel

In [15]:
image_copy = contrived_image.copy()
image_copy[0,0,2] = 42
contrived_image.item((0,0,2)), image_copy.item((0,0,2))

(2.33, 42.0)

In [16]:
image_copy[:1, :1]

array([[[ 3.72,  4.9 , 42.  ]]])

In [17]:
image_copy = contrived_image.copy()
image_copy.itemset((0,0,2), 42)
contrived_image.item((0,0,2)), image_copy.item((0,0,2))

(2.33, 42.0)

In [18]:
image_copy[:1, :1]

array([[[ 3.72,  4.9 , 42.  ]]])

### Create data with NumPy and display

#### Use uniformly distributed data

In [19]:
contrived_image = np.random.rand(height,width,channels)
show_image("Image", contrived_image)

#### Use normally distributed data

In [20]:
contrived_image = np.random.randn(height,width,channels)
show_image("Image", contrived_image)

## Modifying Images
### First, change value of colours deterministically

In [21]:
resized_image = image_resize(input_image)

Enter the scale factor by which to reduce the image size:90


### 1. Reflect on Diagonal

In [44]:
output_image = resized_image.copy()
dim = output_image.shape[0]
output_image = output_image[:dim,:dim,:3]
for r in range(output_image.shape[0]):
    for c in range(output_image.shape[1]):
        
        if(is_outside(r, c, resized_image.shape)):
            output_image.itemset((r,c,0), 0)
            output_image.itemset((r,c,1), 0)
            output_image.itemset((r,c,2), 0)
        else:
            val_0 = output_image.item((c,r,0))
            val_1 = output_image.item((c,r,1))
            val_2 = output_image.item((c,r,2))

        output_image[r,c] = (val_0,val_1,val_2)
show_image("Image", output_image)

### 2. A crude way to detect edges? Maybe?

In [33]:
output_image = resized_image.copy()
diameter = 20

for r in range(resized_image.shape[0]):
    for c in range(resized_image.shape[1]):
        
        if resized_image.item((r,c,1)) >= 128:
            r1 = 0
        else:
            r1 = 255
        if resized_image.item((r,c,1)) >= 128:
            c1 = 0
        else:
            c1 = 255
                
        if(is_outside(r1, c1, resized_image.shape)):
            output_image.itemset((r,c,0), 0)
            output_image.itemset((r,c,1), 0)
            output_image.itemset((r,c,2), 0)
        else:
            output_image.itemset((r,c,0), resized_image.item((r1,c1,0)))
            output_image.itemset((r,c,1), resized_image.item((r1,c1,1)))
            output_image.itemset((r,c,2), resized_image.item((r1,c1,2)))
show_image("Image", output_image)

### 3. Replace values with NumPy generated values

In [41]:
for r in range(resized_image.shape[0]):
    for c in range(resized_image.shape[1]):
        if(is_outside(r, c, resized_image.shape)):
            output_image.itemset((r,c,0), 0)
            output_image.itemset((r,c,1), 0)
            output_image.itemset((r,c,2), 0)
        else:
            output_image.itemset((r,c,0), np.min([resized_image.item((r,c,1)), resized_image.item((r,c,2))]))
            output_image.itemset((r,c,1), np.min([resized_image.item((r,c,0)), resized_image.item((r,c,2))]))
            output_image.itemset((r,c,2), np.min([resized_image.item((r,c,0)), resized_image.item((r,c,1))]))
show_image("Image", output_image)

### 4. Swap around the RGB Channels
That time when you needed to read up on [Permutations and Combinations](https://betterexplained.com/articles/easy-permutations-and-combinations/) and then a [LinkedIn post](https://www.linkedin.com/posts/walidhossain99_learning-teaching-probability-activity-6814662003924332544-OJNZ/) on the same showed up!

In [56]:
output_image = resized_image.copy()
for r in range(output_image.shape[0]):
    for c in range(output_image.shape[1]):
        
        if(is_outside(r, c, resized_image.shape)):
            output_image.itemset((r,c,0), 0)
            output_image.itemset((r,c,1), 0)
            output_image.itemset((r,c,2), 0)
        else:
            val_0 = output_image.item((r,c,0))
            val_1 = output_image.item((r,c,1))
            val_2 = output_image.item((r,c,2))
        output_image[r,c] = (val_0,val_1,val_2)
show_image("Image", output_image)

### 5. Change pixel values using mathematical functions

In [66]:
output_image = resized_image.copy()

for r in range(resized_image.shape[0]):
    for c in range(resized_image.shape[1]):
        output_image.itemset((r,c,0), resized_image.item((r,c,0))*math.e)
        output_image.itemset((r,c,1), resized_image.item((r,c,1))*math.e)
        output_image.itemset((r,c,2), resized_image.item((r,c,2))*math.e)

show_image("Image", output_image)

### 6 Splatter Effect

### Script from [George Lecakes](https://www.linkedin.com/in/george-lecakes-jr-phd-1831814b/) [demonstrating the splatter effect](https://www.youtube.com/watch?v=iLoMqAE_8A8)
#### Using Uniformly Distributed Random Data

In [27]:
output_image = resized_image.copy()
diameter = 20

for r in range(resized_image.shape[0]):
    for c in range(resized_image.shape[1]):
        
        r1 = r + math.ceil(np.random.uniform(-0.5,0.5) * diameter)
        c1 = c + math.ceil(np.random.uniform(-0.5,0.5) * diameter)
                
        if(is_outside(r1, c1, resized_image.shape)):
            output_image.itemset((r,c,0), 0)
            output_image.itemset((r,c,1), 0)
            output_image.itemset((r,c,2), 0)
        else:
            output_image.itemset((r,c,0), resized_image.item((r1,c1,0)))
            output_image.itemset((r,c,1), resized_image.item((r1,c1,1)))
            output_image.itemset((r,c,2), resized_image.item((r1,c1,2)))
show_image("Image", output_image)

#### Using Normally Distributed Random Data

In [28]:
output_image = resized_image.copy()
diameter = 60

for r in range(resized_image.shape[0]):
    for c in range(resized_image.shape[1]):
        
        r1 = r + math.ceil(np.random.normal(-0.5,0.5) * diameter)
        c1 = c + math.ceil(np.random.normal(-0.5,0.5) * diameter)
                
        if(is_outside(r1, c1, resized_image.shape)):
            output_image.itemset((r,c,0), 0)
            output_image.itemset((r,c,1), 0)
            output_image.itemset((r,c,2), 0)
        else:
            output_image.itemset((r,c,0), resized_image.item((r1,c1,0)))
            output_image.itemset((r,c,1), resized_image.item((r1,c1,1)))
            output_image.itemset((r,c,2), resized_image.item((r1,c1,2)))
show_image("Image", output_image)